Copyright (c) Alliance for Sustainable Energy, LLC and Skye Analytics, Inc. See also https://github.com/NREL/ATB-calc/blob/main/LICENSE

# Process an ATB electricity technology
This Jupyter notebook is an example of how to run the ATB data processing pipeline and extract data from the ATB electricity data workbook for an individual technology. The path to the workbook on your computer needs to be updated as indicated below. The notebook will otherwise run as-is. There is commented-out code in several locations showing alternative ways to run the pipeline. 

In [ ]:
import os
import sys
import pandas as pd
from datetime import datetime as dt

sys.path.insert(0, os.path.dirname(os.getcwd()))

# Electricity technology processors
from lcoe_calculator.tech_processors import (
    FixedOffShoreWindProc, FloatingOffShoreWindProc, LandBasedWindProc, DistributedWindProc,
    UtilityPvProc, CommPvProc, ResPvProc, UtilityPvPlusBatteryProc,
    CspProc, GeothermalProc, HydropowerProc, PumpedStorageHydroProc,
    PumpedStorageHydroOneResProc,
    CoalProc, NaturalGasProc, NuclearProc, BiopowerProc,
    UtilityBatteryProc, CommBatteryProc, ResBatteryProc,
    CoalRetrofitProc, NaturalGasRetrofitProc, NaturalGasFuelCellProc
)

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

## Load the ATB electricity workbook
Update the path for `atb_electricity_workbook` to point to your copy of the electricity workbook downloaded from the [ATB website](https://atb.nrel.gov).


In [ ]:
# The below line MUST be updated to reflect the location of the ATB workbook on your computer
atb_electricity_workbook = 'path/to/2024-ATB-Data_Workbook.xlsx'

## Select CRP and financial case and technology

In [ ]:
# The CRP may be one of 20, 30, or 'TechLife' in quotes
crp = 20
# crp = 30
# crp = 'TechLife'

# The financial case can be either 'R&D' or 'Market'
financial_case = 'R&D'
# financial_case = 'Market'

# Finally, choose a technology processor from the list imported from lcoe_calculator.tech_processors 
# above and initialize it. Several options are shown in comments, but only one should be used at a time. 
# Microsoft Excel may open and ask for permission to view the workbook. 
proc = LandBasedWindProc(atb_electricity_workbook, crp=crp, case=financial_case)

# proc = NuclearProc(atb_electricity_workbook, crp=crp, case=financial_case)
# proc = UtilityPvProc(atb_electricity_workbook, crp=crp, case=financial_case)
# proc = NaturalGasProc(atb_electricity_workbook, crp=crp, case=financial_case)
# proc = NuclearProc(atb_electricity_workbook, crp=crp, case=financial_case)

### View techonology meta-data

In [ ]:
proc.get_meta_data()

## Run the processor and test the calculated LCOE and CAPEX against the values in the workbook. 
Note that not all technologies have LCOE and CAPEX. 

In [ ]:
# Run the pipeline
proc.run()

# Test LCOE and CAPEX
proc.test_capex()
proc.test_lcoe()

## View results
Various metrics are available on the technology processors once they have been `run()`. Note that not all technologies have all metrics. Several typical metrics are shown below. The full list of available metrics can be determined by viewing the code in `lcoe_calculator/base_processor.py` and the technology of interest in `lcoe_calculator/tech_processors.py`. Note that all metrics are stored in pandas data frames, as indicated by the `df_` prefix. Any metric may be copied to the clipboard (e.g., for viewing in a spreadsheet) with the `.to_clipboar()` method. For example: `proc.df_lcoe.to_clipboard()`. 

In [ ]:
# LCOE
proc.df_lcoe

In [ ]:
# Net capacity factor
proc.df_ncf

In [ ]:
# WACC
proc.df_wacc

In [ ]:
# Construction finance factor
proc.df_cff

In [ ]:
# Plot LCOE by year for Offshore Wind - Class 1/Advanced
ax = proc.df_lcoe.loc['Offshore Wind - Class 1/Advanced'].plot()
ax.set_xlabel("Year")
ax.set_ylabel("LCOE ($/MWh)")